In [10]:
!jt -t grade3

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics

In [16]:
df=pd.read_csv("Data/ForestRomania.csv")

In [3]:
df.head()

,Category,Region,Year,Unit,Value
0,Total,TOTAL,1990,Thousands hectares,6371.0
1,Total,TOTAL,1991,Thousands hectares,6367.6
2,Total,TOTAL,1992,Thousands hectares,6368.2
3,Total,TOTAL,1993,Thousands hectares,6366.9
4,Total,TOTAL,1994,Thousands hectares,6369.0


In [14]:
df.dtypes

Category     object
Region       object
Year          int64
Unit         object
Value       float64
dtype: object

# Data analysis and preprocessing 

In [19]:
df.isna().sum() #number of na values

Category     0
Region      11
Year         0
Value        0
dtype: int64

In [22]:
df=df.fillna(method='ffill')

In [23]:
df.isnull().sum()

Category    0
Region      0
Year        0
Value       0
dtype: int64

In [7]:
df["Category"].nunique() #number of unique values in category

5

In [11]:
df["Region"].nunique()

55

In [13]:
df["Unit"].nunique()

1

In [18]:
df=df.drop("Unit",axis=1) #dropping Unit feature as it has only one value

# Converting non-numerical columns 
## Features to be modified:
> - Category
> - Region 

In [31]:
# checking number of columns

len(df.columns)

4

# Importing the algortihms:
> - Linear Regression
> - Decision Regressor
> - Random Forest Regressor

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [28]:
#instantiating the models

lr = LinearRegression()
dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()

# Split into train and test

In [25]:
#creating X and Y(target)

X=df.drop("Value",axis=1)
Y=df["Value"]

In [26]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=99)

# Linear Regression

In [29]:
lr.fit(X_train,Y_train)

ValueError: could not convert string to float: 'Wood'